In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [88]:
df = pd.read_csv('./smartphone_cleaned_v5.csv')

In [137]:
df.columns

Index(['brand_name', 'model', 'price', 'rating', 'has_5g', 'has_nfc',
       'has_ir_blaster', 'processor_brand', 'num_cores', 'processor_speed',
       'battery_capacity', 'fast_charging_available', 'fast_charging',
       'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate',
       'resolution', 'num_rear_cameras', 'num_front_cameras', 'os',
       'primary_camera_rear', 'primary_camera_front',
       'extended_memory_available', 'extended_upto', 'pixels'],
      dtype='object')

In [148]:
np.percentile(df['pixels'],20)

1152000.0

In [91]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression.replace(" ", "").replace("x", "*")
        result = eval(expression)
        return 10 if result > 2590000 else 5
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    
    
def categorize_phone(row):
    condition_fast_processor = row["processor_speed"]*10/max_speed
    condition_fluid_display = 10 if row["refresh_rate"] >60 else 5
    condition_many_cores = row["num_cores"]*10/max_cores
    condition_nfc = 5 if row["has_nfc"]==True  else 0
    condition_5g=5 if row["has_5g"]==True else 0
    condition_resolution = 8 + row['pixels']/max_resolution if row['pixels']>=
    total = condition_5g+condition_fluid_display +  condition_fast_processor + condition_nfc + condition_many_cores +condition_resolution
    if total > 40: return "flagship"
    elif total >30: return "midrange"
    else: return "average"
    # no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
    
def categorize_camera(row):
    if row["phone_category"] == "flagship":
        c1 = 10
    elif row["phone_category"] == "midrange":
        c1 = 8
    else:
        c1 = 5

    c2 = (8 + row["primary_camera_rear"]*2/max_rear) if row["primary_camera_rear"] > 12 else 8*row["primary_camera_rear"]/12
    c3 = (8 + row["primary_camera_rear"]*2/max_front) if row["primary_camera_rear"] > 10 else 8*row["primary_camera_rear"]/10
    
    brand_quality = {'apple':5, 'google':5, 'samsumg':4,'huawei':4}
    c4 = brand_quality[row['brand_name']] if row["brand_name"] in brand_quality else 3

    total = 10 + 10 + 10 + 5 # sum of maximum of c1 + c2 + c3 + c4
    rating = (c1+c2+c3)*10.0/total
    return rating
    
def categorize_performance(row):
    c1 = row["processor_speed"]*10/max_speed
    c2 = row["num_cores"]*10/max_cores
    c3 = (8 + row["ram_capacity"]*2/max_ram) if row["ram_capacity"] > 8 else row["ram_capacity"]/8  
    
    total = 10 + 10 + 10
    rating = (c1+c2+c3)*10.0/total
    return rating

In [119]:
x = 8*8/12
x

5.333333333333333

In [ ]:
#defining the ceiling
max_cores = max(df2["num_cores"].unique())
max_speed = max(df2["processor_speed"].unique())
max_ram = max(df2["ram_capacity"].unique())
max_price = max(df2["price"].unique())
max_rear = max(df2["primary_camera_rear"].unique())
max_front = max(df2["primary_camera_front"].unique())
max_resolution = max(df2["pixels"].unique())
max_refresh = max(df2["refresh_rate"].unique())

max_specs = [max_cores,max_speed,max_ram,max_price,max_rear,max_front,max_front,max_resolution,max_refresh]

#defining the ceilings
q1_cores = np.percentile(df['num_cores'].unique(),25)
q1_speed = np.percentile(df2["processor_speed"].unique(),25)
q1_ram = np.percentile(df2["ram_capacity"].unique(),25)
q1_price = np.percentile(df2["price"].unique(),25)
q1_rear = np.percentile(df2["primary_camera_rear"].unique(),25)
q1_front = np.percentile(df2["primary_camera_front"].unique(),25)
q1_resolution = np.percentile(df2["pixels"].unique(),25)
q1_refresh = np.percentile(df2["refresh_rate"].unique(),25)

q1_specs = [q1_cores,q1_speed,q1_ram,q1_price,q1_rear,q1_front,q1_front,q1_resolution,q1_refresh]

#defining the ceiling
q2_cores = np.percentile(df2["num_cores"].unique(),50)
q2_speed = np.percentile(df2["processor_speed"].unique(),50)
q2_ram = np.percentile(df2["ram_capacity"].unique(),50)
q2_price = np.percentile(df2["price"].unique(),50)
q2_rear = np.percentile(df2["primary_camera_rear"].unique(),50)
q2_front = np.percentile(df2["primary_camera_front"].unique(),50)
q2_resolution = np.percentile(df2["pixels"].unique(),50)
q2_refresh = np.percentile(df2["refresh_rate"].unique(),50)

q2_specs = [q2_cores,q2_speed,q2_ram,q2_price,q2_rear,q2_front,q2_front,q2_resolution,q2_refresh]

#defining the ceiling
q3_cores = np.percentile(df2["num_cores"].unique(),75)
q3_speed = np.percentile(df2["processor_speed"].unique(),75)
q3_ram = np.percentile(df2["ram_capacity"].unique(),75)
q3_price = np.percentile(df2["price"].unique(),75)
q3_rear = np.percentile(df2["primary_camera_rear"].unique(),75)
q3_front = np.percentile(df2["primary_camera_front"].unique(),75)
q3_resolution = np.percentile(df2["pixels"].unique(),75)
q3_refresh = np.percentile(df2["refresh_rate"].unique(),75)

q3_specs = [q3_cores,q3_speed,q3_ram,q3_price,q3_rear,q3_front,q3_front,q3_resolution,q3_refresh]



In [92]:
# condition_fast_processor = df["processor_speed"]>3
# condition_fluid_display = df["refresh_rate"]>60
# condition_many_cores = df["num_cores"]>=8
# condition_nfc = df["has_nfc"]==True
# condition_5g=df["has_5g"]==True
# condition_resolution = df["resolution"].apply(evaluate_resolution)
# no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
# # all_conditions = condition_fluid_display &  condition_fast_processor & condition_nfc & condition_5g & condition_many_cores & condition_resolution
# # most_conditions = 
# # average_conditions = 
# # better_than_no_conditions = 
# # no_conditions = 
# conditions = no_of_conditions_satisfied == 6

In [93]:
from copy import deepcopy
df2 = deepcopy(df)
df2.head()
final_rec = deepcopy(df)
final_rec.head()


,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,6.70,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,6.59,120,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,6.60,90,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,6.55,120,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,6.70,120,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN


In [94]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression['resolution'].replace(" ", "").replace("x", "*")
        return  eval(expression)
        
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    # return expression['resolution']
    
df['pixels'] = df.apply(evaluate_resolution,axis=1)
np.percentile(df['pixels'],99)

4631040.0

In [95]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression.replace(" ", "").replace("x", "*")
        return eval(expression)
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False

max_cores = max(df["num_cores"].unique())
max_speed = max(df["processor_speed"].unique())
max_price = max(df["price"].unique())
max_rear = max(df["primary_camera_rear"].unique())
max_front = max(df["primary_camera_front"].unique())
max_resolution = max(df["pixels"].unique())
max_refresh = max(df["refresh_rate"].unique())

max_specs = [max_cores,max_speed,max_price,max_rear,max_front,max_front,max_resolution,max_refresh]
max_specs

[8.0, 3.22, 650000, 200.0, 60.0, 60.0, 6312960, 240]

In [96]:
item = 'price'
p1 = np.percentile(df[item],1)
p2 = np.percentile(df[item],10)
p3 = np.percentile(df[item],90)
p4 = np.percentile(df[item],99)
p1,p2,p3,p4


(5449.89, 8999.0, 69999.0, 165034.2000000002)

In [97]:
# pre-processing

#price
category = 'primary_camera_rear'
p1 = np.percentile(df[category],10)
p2 = np.percentile(df[category],90)
df2 = df.query(f"{category} >= {p1} and {category} <= {p2}")
max(df2[category])
# min(df[category].unique())


64.0

In [98]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression['resolution'].replace(" ", "").replace("x", "*")
        return  eval(expression)
        
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    # return expression['resolution']
    
df2['pixels'] = df2.apply(evaluate_resolution,axis=1)
np.percentile(df2['pixels'],99)

C:\Users\GlitchyStar\AppData\Local\Temp\ipykernel_27416\2833832334.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pixels'] = df2.apply(evaluate_resolution,axis=1)


4631040.0

In [99]:
sorted(df2['pixels'].unique(),reverse=True)

[6312960,
 5456000,
 4631040,
 4608000,
 4561920,
 4492800,
 4435200,
 4377600,
 4262400,
 4147200,
 4138560,
 4134240,
 3942912,
 3903744,
 3725568,
 3606840,
 3566952,
 3528000,
 3477600,
 3440640,
 3439800,
 3338496,
 3170592,
 3168000,
 3013524,
 2962440,
 2880000,
 2851200,
 2822400,
 2767680,
 2764800,
 2721600,
 2678400,
 2656800,
 2643840,
 2604960,
 2600640,
 2596320,
 2592000,
 2579040,
 2566080,
 2527200,
 2501280,
 2462400,
 2425680,
 2397600,
 2332800,
 2073600,
 1483776,
 1209600,
 1188000,
 1185000,
 1180800,
 1160640,
 1152000,
 1123200,
 1111680,
 1094400,
 1036800,
 1000500,
 921600]

In [100]:
expensives = df.query("price <= 5449")
expensives.sort_values(by='price', ascending=True)

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,pixels
189,realme,Realme C2s,3499,63.0,False,False,False,helio,8.0,2.00,...,60,720 x 1560,2,1.0,android,13.0,5.0,1,128.0,1123200
891,lyf,Lyf Earth 2,3890,NaN,False,False,False,snapdragon,8.0,1.50,...,60,1080 x 1920,1,1.0,android,13.0,13.0,1,64.0,2073600
473,lyf,Lyf Earth 1,3990,60.0,False,False,False,snapdragon,8.0,1.50,...,60,1080 x 1920,2,1.0,android,13.0,5.0,1,64.0,2073600
874,itel,Itel A23 Pro,3999,NaN,False,False,False,spreadtrum,4.0,1.40,...,60,854 x 480,1,1.0,android,5.0,2.0,0,NaN,409920
143,jio,Jio Phone 3,4499,NaN,False,False,False,NaN,4.0,1.40,...,60,720 x 1280,1,1.0,android,5.0,2.0,1,128.0,921600
350,jio,Jio JioPhone Next,4649,NaN,False,False,False,snapdragon,4.0,1.30,...,60,720 x 1440,1,1.0,android,13.0,8.0,1,512.0,1036800
972,itel,itel A23s,4787,NaN,False,False,False,spreadtrum,4.0,1.40,...,60,854 x 480,1,NaN,android,2.0,NaN,0,NaN,409920
827,samsung,Samsung Galaxy A01 Core,4999,NaN,False,False,False,mediatek,4.0,1.50,...,60,720 x 1480,1,1.0,android,8.0,5.0,1,NaN,1065600
716,nokia,Nokia C01 Plus,5249,NaN,False,False,False,unisoc,8.0,1.60,...,60,720 x 1440,1,1.0,android,5.0,5.0,1,256.0,1036800
155,realme,Realme C30,5299,NaN,False,False,False,tiger,8.0,1.82,...,60,720 x 1600,1,1.0,android,8.0,5.0,1,1024.0,1152000


In [101]:
objects = df[df["rating"].isnull()]
print(objects)


from copy import deepcopy
df2 = deepcopy(df)
df2.head()

max_cores = max(df["num_cores"].unique())
max_speed = max(df["processor_speed"].unique())

print(max_cores,max_speed)

# Assuming df is your DataFrame
df2['phone_category'] = df2.apply(categorize_phone, axis=1)
df2['camera_category'] = df2.apply(categorize_camera, axis=1)

objects = df2[df["rating"].isnull()]
objects



# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

df2.head()


    brand_name                        model   price  rating  has_5g  has_nfc  \
14     samsung  Samsung Galaxy S23 Ultra 5G  114990     NaN    True     True   
29     oneplus               OnePlus 11 Pro   69999     NaN    True     True   
37     samsung  Samsung Galaxy S22 Ultra 5G   91999     NaN    True     True   
49     samsung        Samsung Galaxy A74 5G   42999     NaN    True     True   
69        oppo             Oppo Find N Fold   99990     NaN    True     True   
..         ...                          ...     ...     ...     ...      ...   
954     huawei                Huawei Mate X  169000     NaN    True     True   
957       vivo                    Vivo Y55S   13490     NaN   False    False   
963       lava                      Lava X3    6999     NaN   False    False   
972       itel                    itel A23s    4787     NaN   False    False   
974       vivo                Vivo X Fold 2  119990     NaN    True     True   

     has_ir_blaster processor_brand  nu

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,pixels,phone_category,camera_category
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,3,1.0,android,50.0,16.0,0,NaN,4631040,midrange,6.0
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,3,1.0,android,64.0,16.0,1,1024.0,2604960,midrange,6.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,3,1.0,android,50.0,13.0,1,1024.0,2600640,midrange,8.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,3,1.0,android,50.0,16.0,1,1024.0,2592000,midrange,6.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,3,1.0,android,108.0,16.0,0,NaN,2604960,midrange,6.0


In [102]:
final_rec = final_rec.query("brand_name == 'apple'")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
6,apple,Apple iPhone 14,65999,81.0,True,True,False,bionic,6.0,3.22,...,6.1,60,1170 x 2532,2,1.0,ios,12.0,12.0,0,NaN
15,apple,Apple iPhone 13,62999,79.0,True,True,False,bionic,6.0,3.22,...,6.1,60,1170 x 2532,2,1.0,ios,12.0,12.0,0,NaN
27,apple,Apple iPhone 14 Pro Max,129990,76.0,True,True,False,bionic,6.0,NaN,...,6.7,120,1290 x 2796,3,1.0,ios,48.0,12.0,0,NaN
33,apple,Apple iPhone 11,38999,73.0,False,True,False,bionic,6.0,2.65,...,6.1,60,828 x 1792,2,1.0,ios,12.0,12.0,0,NaN
56,apple,Apple iPhone 14 Plus,74999,82.0,True,True,False,bionic,6.0,3.22,...,6.7,60,1284 x 2778,2,1.0,ios,12.0,12.0,0,NaN


In [103]:
final_rec = final_rec.query("price >= 100000")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
27,apple,Apple iPhone 14 Pro Max,129990,76.0,True,True,False,bionic,6.0,NaN,...,6.70,120,1290 x 2796,3,1.0,ios,48.0,12.0,0,NaN
76,apple,Apple iPhone 13 Pro,119900,83.0,True,True,False,bionic,6.0,3.22,...,6.10,120,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN
100,apple,Apple iPhone 14 Pro,119990,75.0,True,True,False,bionic,6.0,NaN,...,6.10,120,1179 x 2556,3,1.0,ios,48.0,12.0,0,NaN
152,apple,Apple iPhone 15 Pro Max,142990,79.0,True,True,False,bionic,NaN,NaN,...,6.68,120,1284 x 2778,3,1.0,ios,50.0,12.0,0,NaN
161,apple,Apple iPhone 13 Pro Max,129900,84.0,True,True,False,bionic,6.0,3.22,...,6.70,120,1284 x 2778,3,1.0,ios,12.0,12.0,0,NaN


In [104]:
final_rec = final_rec.query("refresh_rate <= 90")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
594,apple,Apple iPhone 11 Pro Max,109900,77.0,False,True,False,bionic,6.0,2.65,...,6.5,60,1242 x 2688,3,1.0,ios,12.0,12.0,0,NaN
613,apple,Apple iPhone 12 Pro (512GB),139900,80.0,True,True,False,bionic,6.0,3.10,...,6.1,60,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN
788,apple,Apple iPhone 14 Plus (512GB),104999,83.0,True,True,False,bionic,6.0,3.22,...,6.7,60,1284 x 2778,2,1.0,ios,12.0,12.0,0,NaN
824,apple,Apple iPhone 12 Pro (256GB),119900,80.0,True,True,False,bionic,6.0,3.10,...,6.1,60,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN


In [105]:
pixels = df.query("brand_name == 'google'")


In [106]:
level5 = ['excellent', 'outstanding','exceptional','remarkable','first-rate','top-notch','exceptionally good',' splendid', 'impressive',"Superlative","Exemplary","Prime","Stellar","Astonishing","Fabulous","Phenomenal","Spectacular","Marvelous","Exceptionally","Incredible","Brilliant","Sublime","Distinguished","Peerless","Unparalleled","Eminent","Unsurpassed","Unbeatable","Epic"]
level1 = adjectives_worst = ["Abysmal","Appalling","Atrocious","Awful","Dreadful","Terrible","Horrendous","Horrible","Deplorable","Disastrous","Disgusting","Execrable","Lousy","Pathetic","Shameful","Unspeakable","Vile","Wretched","Miserable","Catastrophic","Grim","Woeful","Dire","Repulsive","Rotten","Detestable","Unacceptable","Inferior","Foul","Unsatisfactory"]
categories = [] #performance,camera,software,build,display,photo,video,video, storage,speaker

In [107]:
stage1 = [
    'How do you want to get recommendation?',
    'Entering your own phrase',
    'Get your questions asked'
]

exact_questions = [
    '1.) IPhone 2.) Android 3.)Open to both',
    "Do you want a particular smartphone brand? \n1.) No 2.) Yes (Input your brand)",
    'What screen size would you prefer? \n1.) below 6.1" 2.) between 6.1" & 6.6" 3.) above 6.6" 4.) Doesn\'t matter',
    'Do you want your phone to have 5g? \n1.) Yes 2.) No 3.) Any',
    'What is your budget range Example (inside the "") "45000-50000"?'
]
performance_questions = [
    'Do you frequently play games on your smartphone? (y/n)',
    'Do you use a  lot of apps? (y/n)',
    'Do you want to take high quality photos and videos (y/n)',
    'Do you wish to use your smartphone for over 5 years? (y/n)',
    'Do you wish to have all new features in your smartphone? (y/n)'
]
camera_questions = [
    'Do you want to take high quality photos and videos? (y/n)',
    'Do you want a camera that shoot good in night time? (y/n)'
    'Would you love to shoot zoom and potrait shots? (y/n)'
    'Do you want to shoot many videos? (y/n)',
    'Do you want to shoot many photos? (y/n)'
]
software_questions = [
    'Do you want to get new updates for 5+ years? (y/n)'
    "Do you mind useless apps pre-installed in your smartphone? (y/n)",
    'Do you like to have easy software support? (y/n)'
    'Can you distinguish between smooth and non smooth animation? (y/n)'
]
display_questions = [
    'Do you do a lot of scrolling? (y/n)',
    "Do you like watching movies in mobile? (y/n)",
    'Do you consume a lot of media like youtube and more? (y/n)',
    'Do you wish to use phone a lot on sunlight? (y/n)'
]
storage_questions = [
    "Do you take a lot of photos and videos? (y/n)",
    'Do you store a lot of offline medias like movies or other files in smartphones? (y/n)',
    "Do you want your smartphone to be able to store everything you want for years to come? (y/n)",
    'Do you want to transfer from and to your smartphones quickly? (y/n)'
    'Do you download a lot of apps orr games? (y/n)',
]
speaker_questions = [
    "Would you like a good speaker? \n1.) yes (y) 2.) don't mind (n)"
]
battery_questions = [
    "Do you like a bigger battery? (y/n)",
    'Do you charge very rarely? (y/n)'
    "Do you want fast charging? (y/n)",
    'Do you want wireless charging? (y/n)',
]

all_categories = [exact_questions,performance_questions, camera_questions,software_questions,display_questions,storage_questions,speaker_questions]

In [108]:
for prompts in stage1:
    print(prompts)
lst = []

How do you want to get recommendation?
Entering your own phrase
Get your questions asked


In [109]:
lst = []
for category in all_categories:
    for questions in category:
        lst.append(input(questions))


In [110]:
df2.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,pixels,phone_category,camera_category
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,3,1.0,android,50.0,16.0,0,NaN,4631040,midrange,6.0
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,3,1.0,android,64.0,16.0,1,1024.0,2604960,midrange,6.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,3,1.0,android,50.0,13.0,1,1024.0,2600640,midrange,8.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,3,1.0,android,50.0,16.0,1,1024.0,2592000,midrange,6.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,3,1.0,android,108.0,16.0,0,NaN,2604960,midrange,6.0
